### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [4]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}{CME}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"


In [5]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)


In [6]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()


In [7]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))



{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [8]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [9]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df.dropna(subset='linkedEvents', inplace=True)
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [10]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []
        
# Iterate over each index in the DataFrame
for i in cme_df.index:
    activityID = cme_df.loc[i, 'activityID'],
    startTime = cme_df.loc[i, 'startTime'],
    linkedEvents = cme_df.loc[i, 'linkedEvents']

    # Iterate over each dictionary in the list
    for c in linkedEvents:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({'activityID': activityID, 'startTime': startTime, 'linkedEvents': c})
# Create a new DataFrame from the expanded rows
expanded_cme_df = pd.DataFrame(expanded_rows)
expanded_cme_df

,activityID,startTime,linkedEvents
0,"(2013-05-01T03:12:00-CME-001,)","(2013-05-01T03:12Z,)",{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,"(2013-05-03T22:36:00-CME-001,)","(2013-05-03T22:36Z,)",{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,"(2013-05-09T19:29:00-CME-001,)","(2013-05-09T19:29Z,)",{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,"(2013-05-13T02:54:00-CME-001,)","(2013-05-13T02:54Z,)",{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,"(2013-05-13T02:54:00-CME-001,)","(2013-05-13T02:54Z,)",{'activityID': '2013-05-13T04:12:00-SEP-001'}
...,...,...,...
1709,"(2024-04-24T01:48:00-CME-001,)","(2024-04-24T01:48Z,)",{'activityID': '2024-04-26T00:17:00-IPS-001'}
1710,"(2024-04-24T15:05:00-CME-001,)","(2024-04-24T15:05Z,)",{'activityID': '2024-04-24T14:02:00-FLR-001'}
1711,"(2024-04-25T18:24:00-CME-001,)","(2024-04-25T18:24Z,)",{'activityID': '2024-04-25T17:03:00-FLR-001'}
1712,"(2024-04-30T00:36:00-CME-001,)","(2024-04-30T00:36Z,)",{'activityID': '2024-04-30T00:46:00-FLR-001'}


In [11]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(expanded_cme_df):
    try:
        # Attempt to access the 'activityID' from the provided dictionary
        return expanded_cme_df['activityID']
    except KeyError:
        # If the key doesn't exist, print an error message and return None
        print("KeyError: 'activityID' not found in the dictionary")
        return None


In [12]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))


In [13]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_cme_df = expanded_cme_df[expanded_cme_df['GST_ActivityID'].notna()]


In [14]:
# print out the datatype of each column in this DataFrame:
print(expanded_cme_df.dtypes)


activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object


In [15]:
# Convert the 'GST_ActivityID' column to string format 
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['GST_ActivityID'].astype(str)
expanded_cme_df['startTime'].head(10)

0    (2013-05-01T03:12Z,)
1    (2013-05-03T22:36Z,)
2    (2013-05-09T19:29Z,)
3    (2013-05-13T02:54Z,)
4    (2013-05-13T02:54Z,)
5    (2013-05-13T16:18Z,)
6    (2013-05-13T16:18Z,)
7    (2013-05-14T01:30Z,)
8    (2013-05-15T02:18Z,)
9    (2013-05-15T02:18Z,)
Name: startTime, dtype: object

In [16]:
# Convert startTime to datetime format  
expanded_cme_df['startTime'] = pd.to_datetime(expanded_cme_df['startTime'], errors='coerce')
expanded_cme_df['startTime'].head(10)
expanded_cme_df['startTime'].apply(type).value_counts()

startTime
<class 'pandas._libs.tslibs.nattype.NaTType'>    1714
Name: count, dtype: int64

In [17]:
# Rename startTime to startTime_CME and activityID to cmeID
expanded_cme_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)
# Drop linkedEvents
expanded_cme_df.drop(columns=['linkedEvents'], inplace=True)
# Verify that all steps were executed correctly
# Check the structure and data types
expanded_cme_df.info()
# Display the first few rows of the modified DataFrame
expanded_cme_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   cmeID           1714 non-null   object        
 1   startTime_CME   0 non-null      datetime64[ns]
 2   GST_ActivityID  1714 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 40.3+ KB


,cmeID,startTime_CME,GST_ActivityID
0,"(2013-05-01T03:12:00-CME-001,)",NaT,2013-05-04T04:52:00-IPS-001
1,"(2013-05-03T22:36:00-CME-001,)",NaT,2013-05-07T04:37:00-IPS-001
2,"(2013-05-09T19:29:00-CME-001,)",NaT,2013-05-12T23:30:00-IPS-001
3,"(2013-05-13T02:54:00-CME-001,)",NaT,2013-05-13T01:53:00-FLR-001
4,"(2013-05-13T02:54:00-CME-001,)",NaT,2013-05-13T04:12:00-SEP-001


In [18]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
expanded_cme_df = expanded_cme_df[expanded_cme_df['GST_ActivityID'].str.contains('GST')]
expanded_cme_df

,cmeID,startTime_CME,GST_ActivityID
21,"(2013-06-02T20:24:00-CME-001,)",NaT,2013-06-07T03:00:00-GST-001
48,"(2013-09-29T22:40:00-CME-001,)",NaT,2013-10-02T03:00:00-GST-001
90,"(2013-12-04T23:12:00-CME-001,)",NaT,2013-12-08T00:00:00-GST-001
148,"(2014-02-16T14:15:00-CME-001,)",NaT,2014-02-19T03:00:00-GST-001
151,"(2014-02-18T01:25:00-CME-001,)",NaT,2014-02-20T03:00:00-GST-001
...,...,...,...
1519,"(2023-12-14T17:38:00-CME-001,)",NaT,2023-12-18T06:00:00-GST-001
1645,"(2024-02-28T17:48:00-CME-001,)",NaT,2024-03-03T18:00:00-GST-001
1665,"(2024-03-23T01:25:00-CME-001,)",NaT,2024-03-24T12:00:00-GST-001
1669,"(2024-03-23T01:48:00-CME-001,)",NaT,2024-03-24T12:00:00-GST-001


### GST Data

In [20]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"


In [21]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)


In [22]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))



{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [23]:
# Convert gst_json to a Pandas DataFrame 
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]


In [24]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df[gst_df['linkedEvents'].notna()]


In [25]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)


In [26]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df['CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df[gst_df['CME_ActivityID'].notna()]


In [27]:
# Convert the 'CME_ActivityID' column to string format 
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
gst_df['gstID'] = gst_df['gstID'].astype(str)

# Convert startTime to datetime format  
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])

# Rename startTime to startTime_GST 
gst_df.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
gst_df.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly


In [28]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME')]


### Merge both datatsets

In [30]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst_df, expanded_cme_df, left_on='CME_ActivityID', right_on='GST_ActivityID')

In [31]:
# Verify that the new DataFrame has the same number of rows as cme and gst
print(f"Number of rows in merged DataFrame: {merged_df.shape[0]}")


Number of rows in merged DataFrame: 0


### Computing the time it takes for a CME to cause a GST

In [34]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = (merged_df['startTime_GST'] - merged_df['startTime_CME']).dt.total_seconds()
merged_df.head()


TypeError: Cannot subtract tz-naive and tz-aware datetime-like objects

In [ ]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
time_metrics = merged_df['timeDiff'].describe()
print(time_metrics)

### Exporting data in csv format

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('cmes_gsts.csv', index=False)
